## Import Dependencies 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import lxml.html as lh
from sklearn.decomposition import PCA
from sklearn import preprocessing
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
import os
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Get Data

List of NBA Teams

In [2]:
TEAMS =[['1610612737', 'ATL'],
 ['1610612738', 'BOS'],
 ['1610612741', 'CHI'],
 ['1610612766', 'CHR'],
 ['1610612739', 'CLE'],
 ['1610612742', 'DAL'],
 ['1610612743' ,'DEN'],
 ['1610612765' ,'DET'],
 ['1610612744', 'GSW'],
 ['1610612745' ,'HOU'],
 ['1610612754' ,'IND'],
 ['1610612746' ,'LAC'],
 ['1610612747' ,'LAL'],
 ['1610612763' ,'MEM'],
 ['1610612748' ,'MIA'],
 ['1610612749' ,'MIL'],
 ['1610612750' ,'MIN'],
 ['1610612751' ,'NJN'],
 ['1610612740' ,'NOH'],
 ['1610612752' ,'NYK'],
 ['1610612753' ,'ORL'],
 ['1610612755' ,'PHI'],
 ['1610612756' ,'PHO'],
 ['1610612757' ,'POR'],
 ['1610612758' ,'SAC'],
 ['1610612759' ,'SAS'],
 ['1610612760' ,'SEA'],
 ['1610612761' ,'TOR'],
 ['1610612762' ,'UTA'],
 ['1610612764' ,'WAS']]

In [3]:
# take out CHR Team (ID 1610612766) as there is no season data for it in 2002
# Team ID 
team_ID = [1610612737, 1610612738, 1610612741, 1610612739, 1610612742, 1610612743, 1610612765, 1610612744, 1610612745, 1610612754, 1610612746, 1610612747, 1610612763, 1610612748, 1610612749, 1610612750, 1610612751, 1610612740, 1610612752, 1610612753, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612764]

##Team Season Data

In [5]:
d = os.path.join("/content/team_season.txt")
team_season = np.loadtxt(d, delimiter=',', dtype='str')
# get header
header = team_season[0]
# remove first row, header
team_season = team_season[1:]
# convert it to an CSV file
pd.DataFrame(team_season).to_csv("team_stats.csv", header = header,index=False)

In [6]:
col_list = ["team", "year",'o_ftm','o_reb' ,'o_asts' ,'o_pf' ,'o_stl' ,'o_to' ,'o_blk', 'o_3pm','o_pts',
 'd_ftm' ,'d_reb', 'd_asts' ,'d_pf',
 'd_stl', 'd_to', 'd_blk','d_3pm','d_pts','won','lost']

team_stats = pd.read_csv("team_stats.csv",usecols=col_list)

#get team stats for 2002
stats_df = team_stats.loc[team_stats["year"] == 2002]
stats_df.head()

,team,year,o_ftm,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_pts,d_ftm,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_pts,won,lost
1099,ATL,2002,1594,3495,1679,1783,611,1304,474,402,7714,1530,3492,1771,1668,738,1020,399,446,8006,35,47
1100,BOS,2002,1480,3320,1575,1758,720,1105,303,719,7599,1560,3687,1824,1798,617,1256,368,463,7631,44,38
1101,CHI,2002,1455,3526,1782,2028,605,1330,461,349,7786,1794,3689,1949,1795,769,1163,413,431,8207,30,52
1102,CLE,2002,1502,3660,1712,1864,636,1430,521,293,7495,1712,3423,1972,1709,802,1112,486,428,8284,17,65
1103,DAL,2002,1486,3456,1837,1730,665,906,449,636,8444,1477,3726,1781,1744,546,1259,318,477,7806,60,22


In [7]:
TeamStats_2002 = np.array(stats_df)
#skip first 2 columns (team name and year)
team_stats = TeamStats_2002[:,2 :] 

## Game Data 

In [9]:
col_list =["HOME_TEAM_ID","VISITOR_TEAM_ID","SEASON", "HOME_TEAM_WINS"]
game_data = pd.read_csv("games.csv", usecols=col_list)
# get games only for season 2003
game_data = game_data.loc[game_data["SEASON"] == 2003]
# take CHR Game data out 
game_data = game_data.loc[game_data["HOME_TEAM_ID"] != 1610612766]
game_data = game_data.loc[game_data["VISITOR_TEAM_ID"] != 1610612766]
game_data.head()


,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
14448,1610612765,1610612747,2003,1
14449,1610612765,1610612747,2003,1
14450,1610612765,1610612747,2003,1
14451,1610612747,1610612765,2003,1
14452,1610612747,1610612765,2003,0


In [10]:
game_data = np.array(game_data)

# Create the Feature Vector

In [11]:
def createDataSet(game_data,usePCA,isNormalized):
  pca = PCA(n_components=18)
  normalizer = preprocessing.MinMaxScaler()
  data = []
  for i in range(len(game_data)):

    # find location home_team stats
    find_index = np.where(team_ID == game_data[i][0])
    index = find_index[0][0]
    # get home_team stats
    home_side = np.array(team_stats[index])

  
    # get away_team stats
    find_index = np.where(team_ID == game_data[i][1])
    index = find_index[0][0]
    away_side =  np.array(team_stats[index])


    # create feature vector concatenate home_team stats and away_team stats
    feature_vector = np.concatenate((home_side, away_side), axis=None)

    data.append(feature_vector)

  #NORMALIZE
  if (isNormalized):
    data = normalizer.fit_transform(data)

  #reduce feature vector
  if (usePCA):
    data = pca.fit_transform(data)

  ndata = []

  for z in range(len(game_data)):
    result = []
    result.append(game_data[z][3])

    # create feature vector by appending the outcome labled result of the game  
    feature_vector = np.concatenate((data[z],result), axis=None)
    ndata.append(feature_vector)

  data = np.array(ndata)
  
  return data

# ML Techniques

Scoring matrix 

In [13]:
def scores(model):
    
    model.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    
    print("Accuracy score: %.3f" % metrics.accuracy_score(ytest, y_pred))
    print("Recall: %.3f" % metrics.recall_score(ytest, y_pred))
    print("Precision: %.3f" % metrics.precision_score(ytest, y_pred))
    print("F1: %.3f" % metrics.f1_score(ytest, y_pred))
    
    proba = model.predict_proba(xtest)
    print("Log loss: %.3f" % metrics.log_loss(ytest, proba))

    pos_prob = proba[:, 1]
    print("Area under ROC curve: %.3f" % metrics.roc_auc_score(ytest, pos_prob))
    
    cv = cross_val_score(model, xtest, ytest, cv = 10, scoring = 'accuracy')
    print("Accuracy (cross validation score): %0.3f (+/- %0.3f)" % (cv.mean(), cv.std() * 2))

    print("confusion matrix")
    print( metrics.confusion_matrix(ytest, y_pred) )
    
    return y_pred

## Training and Testing our Models

Get the different Data Sets

In [14]:
#Original data
dA = createDataSet(game_data,False,False)
#PCA with Normalized Data
dB = createDataSet(game_data,True,True)
#PCA Data
dC = createDataSet(game_data,True,False)
#Normalized Data
dD = createDataSet(game_data,False,True)

**Results using Original Data (NO PCA or Normalization)**

In [45]:
trainA, testA = train_test_split( dA, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 

print("Training set size: %.0f" % len(xtrain))
print("Testing set size: %.0f" % len(xtest))
print()

print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',C=100,probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

Training set size: 1038
Testing set size: 347

SVM STATS
Accuracy score: 0.637
Recall: 0.847
Precision: 0.624
F1: 0.719
Log loss: 0.641
Area under ROC curve: 0.684
Accuracy (cross validation score): 0.617 (+/- 0.124)
confusion matrix
[[ 60  97]
 [ 29 161]]

knn STATS
Accuracy score: 0.588
Recall: 0.726
Precision: 0.603
F1: 0.659
Log loss: 0.669
Area under ROC curve: 0.621
Accuracy (cross validation score): 0.608 (+/- 0.151)
confusion matrix
[[ 66  91]
 [ 52 138]]

logreg STATS
Accuracy score: 0.622
Recall: 0.805
Precision: 0.619
F1: 0.700
Log loss: 0.651
Area under ROC curve: 0.670
Accuracy (cross validation score): 0.596 (+/- 0.123)
confusion matrix
[[ 63  94]
 [ 37 153]]


**Results using PCA Data**

In [38]:
trainA, testA = train_test_split( dC, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.643
Recall: 0.868
Precision: 0.625
F1: 0.727
Log loss: 0.651
Area under ROC curve: 0.655
Accuracy (cross validation score): 0.614 (+/- 0.122)
confusion matrix
[[ 58  99]
 [ 25 165]]

knn STATS
Accuracy score: 0.591
Recall: 0.716
Precision: 0.607
F1: 0.657
Log loss: 0.675
Area under ROC curve: 0.615
Accuracy (cross validation score): 0.614 (+/- 0.135)
confusion matrix
[[ 69  88]
 [ 54 136]]

logreg STATS
Accuracy score: 0.622
Recall: 0.842
Precision: 0.613
F1: 0.710
Log loss: 0.657
Area under ROC curve: 0.657
Accuracy (cross validation score): 0.588 (+/- 0.145)
confusion matrix
[[ 56 101]
 [ 30 160]]


**Results using Normalized Data**

In [44]:
trainA, testA = train_test_split( dD, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.637
Recall: 0.853
Precision: 0.623
F1: 0.720
Log loss: 0.652
Area under ROC curve: 0.661
Accuracy (cross validation score): 0.637 (+/- 0.180)
confusion matrix
[[ 59  98]
 [ 28 162]]

knn STATS
Accuracy score: 0.608
Recall: 0.800
Precision: 0.608
F1: 0.691
Log loss: 0.668
Area under ROC curve: 0.620
Accuracy (cross validation score): 0.616 (+/- 0.131)
confusion matrix
[[ 59  98]
 [ 38 152]]

logreg STATS
Accuracy score: 0.625
Recall: 0.826
Precision: 0.618
F1: 0.707
Log loss: 0.643
Area under ROC curve: 0.682
Accuracy (cross validation score): 0.616 (+/- 0.150)
confusion matrix
[[ 60  97]
 [ 33 157]]


**Results using PCA with Normalized Data**

In [ ]:
trainA, testA = train_test_split( dB, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.631
Recall: 0.858
Precision: 0.617
F1: 0.718
Log loss: 0.655
Area under ROC curve: 0.654
Accuracy (cross validation score): 0.645 (+/- 0.175)
confusion matrix
[[ 56 101]
 [ 27 163]]

knn STATS
Accuracy score: 0.599
Recall: 0.800
Precision: 0.601
F1: 0.686
Log loss: 0.672
Area under ROC curve: 0.609
Accuracy (cross validation score): 0.617 (+/- 0.114)
confusion matrix
[[ 56 101]
 [ 38 152]]

logreg STATS
Accuracy score: 0.625
Recall: 0.847
Precision: 0.615
F1: 0.712
Log loss: 0.650
Area under ROC curve: 0.666
Accuracy (cross validation score): 0.605 (+/- 0.151)
confusion matrix
[[ 56 101]
 [ 29 161]]
